In [5]:
# From here: https://stackoverflow.com/questions/49264194/import-py-file-in-another-directory-in-jupyter-notebook
import sys
import os

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)


NameError: name '__file__' is not defined

In [2]:
import src.mcmc as mcmc
import src.utils as utils
import numpy as np
from src.ansatz import RBM
from src.model import Model
from config_parser import get_config_file

import matplotlib.pyplot as plt

In [4]:
print(module_path)

parameters = get_config_file()['parameters']


visible_layer_size = parameters['visible_size']  # Number of qubits
hidden_layer_size = parameters['hidden_size']  # Number of hidden nodes

C:\Users\overs\Python\Master-Thesis-Project


FileNotFoundError: [Errno 2] No such file or directory: 'config.json'

In [9]:
seed = 44  # Seed for random number generator
np.random.seed(seed)

b = utils.random_complex_array(visible_layer_size)  # Visible layer bias
c = utils.random_complex_array(hidden_layer_size)  # Hidden layer bias
W = utils.random_complex_matrix(visible_layer_size, hidden_layer_size)  # Visible - hidden weights
#H = random_hamiltonian(2**visible_layer_size)  # Hamiltonian

#H =np.array([[-2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 2]])
H = np.diag([-2, 0, 2, 4])  # Hamiltonian

In [12]:
walker = mcmc.Walker()
rbm = RBM(visible_bias=b, hidden_bias=c, weights=W)  # Initializing RBM currently with random configuration and parameters
model = Model(rbm, walker, H)  # Initializing model with RBM and Hamiltonian

model.walker.estimate_distribution(model.rbm.probability)  # Estimate the distribution

In [13]:
print(f"Accept rate: {model.walker.average_acceptance()}")
print(f"Estimated energy: {model.estimate_energy()}")
print(f"Exact energy: {np.linalg.eigvalsh(H)}")

Accept rate: 0.573
Estimated energy: 0.312
Exact energy: [-2.  0.  2.  4.]


In [14]:
model.gradient_descent_2()
model.gradient_descent_1()
print(f"Estimated energy: {model.estimate_energy()}")

Gradient descent step 0, energy: (0.3386240690401453+0j)
Params 2: [-0.70061478+1.24614003j  1.36635732-1.60491574j -1.51814362-0.0523222j
 -1.66507052+0.55547164j  1.80878379-0.96340369j  0.13758794-0.18032147j
 -1.13340923-1.19050362j  0.65544587+0.21987182j -1.00165039-0.21291913j
  0.31085629-1.41149914j  1.11061024-0.48298102j -0.06715219+1.20176208j
  0.87566353-0.70580303j -1.15981356+0.02518135j]
Gradient descent step 1, energy: (0.25227367624976405+0j)
Params 2: [-0.65055639+1.24614003j  1.41641221-1.60491574j -1.56162951-0.0523222j
 -1.61671419+0.55547164j  1.75882917-0.96340369j  0.18736528-0.18032147j
 -1.08336271-1.19050362j  0.70550961+0.21987182j -1.05162863-0.21291913j
  0.26677527-1.41149914j  1.1606699 -0.48298102j -0.01710079+1.20176208j
  0.92551116-0.70580303j -1.10977761+0.02518135j]
Gradient descent step 2, energy: (0.16992003826925428+8.84760105397744e-18j)
Params 2: [-0.6014872 +1.24614003j  1.46601831-1.60491574j -1.60742686-0.0523222j
 -1.57001019+0.55547164j

Params 2: [ 0.30889229+1.24614003j  1.56930761-1.60491574j -1.4562074 -0.0523222j
 -1.94508981+0.55547164j  1.93338972-0.96340369j  0.47507752-0.18032147j
 -0.1969061 -1.19050362j  1.5112651 +0.21987182j -0.67802713-0.21291913j
  1.11184746-1.41149914j  1.51840811-0.48298102j  0.09953693+1.20176208j
  0.03562508-0.70580303j -1.10268189+0.02518135j]
Gradient descent step 21, energy: (-0.4088740746845547+0j)
Params 2: [ 0.35734584+1.24614003j  1.52376894-1.60491574j -1.46844737-0.0523222j
 -1.9985803 +0.55547164j  1.95658996-0.96340369j  0.42677957-0.18032147j
 -0.16761446-1.19050362j  1.55812364+0.21987182j -0.64548566-0.21291913j
  1.14912212-1.41149914j  1.47930851-0.48298102j  0.05480308+1.20176208j
 -0.02173924-0.70580303j -1.15303143+0.02518135j]
Gradient descent step 22, energy: (-0.405234473068996+0j)
Params 2: [ 0.40439233+1.24614003j  1.47693447-1.60491574j -1.48850298-0.0523222j
 -2.05255953+0.55547164j  1.97622789-0.96340369j  0.37652837-0.18032147j
 -0.14110419-1.19050362j  

Params 2: [ 1.00525593+1.24614003j  0.76764258-1.60491574j -2.26073297-0.0523222j
 -2.85786768+0.55547164j  1.78811742-0.96340369j -0.46535163-0.18032147j
  0.17081462-1.19050362j  2.34705539+0.21987182j -0.47058316-0.21291913j
  1.62563845-1.41149914j  0.71759185-0.48298102j -0.67166148+1.20176208j
 -0.96575626-0.70580303j -1.99125665+0.02518135j]
Gradient descent step 41, energy: (-0.23256492769588613-2.31327981259811e-17j)
Params 2: [ 1.02534737+1.24614003j  0.74370983-1.60491574j -2.29915972-0.0523222j
 -2.88669074+0.55547164j  1.76418578-0.96340369j -0.49629575-0.18032147j
  0.18448711-1.19050362j  2.3745873 +0.21987182j -0.47158769-0.21291913j
  1.64264464-1.41149914j  0.69095628-0.48298102j -0.69449136+1.20176208j
 -1.00206936-0.70580303j -2.01843967+0.02518135j]
Gradient descent step 42, energy: (-0.22470902589780783+2.10434286627502e-17j)
Params 2: [ 1.04439905+1.24614003j  0.72130332-1.60491574j -2.33606194-0.0523222j
 -2.91383095+0.55547164j  1.73984861-0.96340369j -0.525452

Params 2: [ 1.24734292+1.24614003j  0.49591558-1.60491574j -2.78642855-0.0523222j
 -3.20493491+0.55547164j  1.32335928-0.96340369j -0.83407598-0.18032147j
  0.36640955-1.19050362j  2.68531178+0.21987182j -0.43259285-0.21291913j
  1.84996398-1.41149914j  0.4007548 -0.48298102j -0.93134012+1.20176208j
 -1.49546776-0.70580303j -2.30220781+0.02518135j]
Gradient descent step 61, energy: (-0.14957435028595673+0j)
Params 2: [ 1.25341004+1.24614003j  0.48973275-1.60491574j -2.80225686-0.0523222j
 -3.21381449+0.55547164j  1.30311444-0.96340369j -0.84325187-0.18032147j
  0.3724079 -1.19050362j  2.69395404+0.21987182j -0.42736765-0.21291913j
  1.85625926-1.41149914j  0.39299645-0.48298102j -0.93728308+1.20176208j
 -1.51370877-0.70580303j -2.30934033+0.02518135j]
Gradient descent step 62, energy: (-0.14790650751273632+1.27033016808472e-17j)
Params 2: [ 1.25913633+1.24614003j  0.48409541-1.60491574j -2.81689098-0.0523222j
 -3.22197157+0.55547164j  1.28436616-0.96340369j -0.85168965-0.18032147j
  0.

Params 2: [ 1.32947011+1.24614003j  0.41983693-1.60491574j -3.01358625-0.0523222j
 -3.32241111+0.55547164j  0.95671422-0.96340369j -0.95403815-0.18032147j
  0.45612338-1.19050362j  2.80296891+0.21987182j -0.30071101-0.21291913j
  1.93938903-1.41149914j  0.30085785-0.48298102j -1.00466845+1.20176208j
 -1.79918009-0.70580303j -2.39041371+0.02518135j]
Gradient descent step 81, energy: (-0.13059588965724522+0j)
Params 2: [ 1.3320899 +1.24614003j  0.41748282-1.60491574j -3.0221658 -0.0523222j
 -3.32646933+0.55547164j  0.9393133 -0.96340369j -0.95808414-0.18032147j
  0.45929758-1.19050362j  2.80673931+0.21987182j -0.29387196-0.21291913j
  1.94240133-1.41149914j  0.29755204-0.48298102j -1.00697169+1.20176208j
 -1.81264322-0.70580303j -2.39314058+0.02518135j]
Gradient descent step 82, energy: (-0.12998893040862447+0j)
Params 2: [ 1.33462532+1.24614003j  0.41518047-1.60491574j -3.03066086-0.0523222j
 -3.33047066+0.55547164j  0.92184055-0.96340369j -0.96204755-0.18032147j
  0.46239212-1.19050362

Params 2: [ 1.36794255+1.24614003j  0.38022224-1.60491574j -3.17044254-0.0523222j
 -3.3949291 +0.55547164j  0.61249314-0.96340369j -1.023741  -0.18032147j
  0.50472392-1.19050362j  2.85834203+0.21987182j -0.17472123-0.21291913j
  1.98693787-1.41149914j  0.24344148-0.48298102j -1.04412748+1.20176208j
 -2.06173248-0.70580303j -2.43590388+0.02518135j]
Gradient descent step 101, energy: (-0.11879272731353552+0j)
Params 2: [ 1.36937485+1.24614003j  0.37856255-1.60491574j -3.17744806-0.0523222j
 -3.39811395+0.55547164j  0.59636329-0.96340369j -1.02673409-0.18032147j
  0.50660549-1.19050362j  2.8603995 +0.21987182j -0.16903762-0.21291913j
  1.98880423-1.41149914j  0.24097531-0.48298102j -1.04580558+1.20176208j
 -2.07390129-0.70580303j -2.43779516+0.02518135j]
Gradient descent step 102, energy: (-0.11825536436391186+0j)
Params 2: [ 1.37080963+1.24614003j  0.37696024-1.60491574j -3.18428168-0.0523222j
 -3.40121319+0.55547164j  0.58070128-0.96340369j -1.02964   -0.18032147j
  0.50849521-1.190503

Params 2: [ 1.39641807+1.24614003j  0.35499196-1.60491574j -3.28420406-0.0523222j
 -3.44560731+0.55547164j  0.35667043-0.96340369j -1.07168915-0.18032147j
  0.54249731-1.19050362j  2.89911235+0.21987182j -0.05811407-0.21291913j
  2.02277643-1.41149914j  0.20564753-0.48298102j -1.06925598+1.20176208j
 -2.25279823-0.70580303j -2.46502932+0.02518135j]
Gradient descent step 121, energy: (-0.11215404005484694-9.355117543830416e-18j)
Params 2: [ 1.39762017+1.24614003j  0.35377425-1.60491574j -3.28973017-0.0523222j
 -3.44806715+0.55547164j  0.34409562-0.96340369j -1.07402241-0.18032147j
  0.54410243-1.19050362j  2.90082886+0.21987182j -0.05306059-0.21291913j
  2.02426871-1.41149914j  0.2038079 -0.48298102j -1.07048623+1.20176208j
 -2.26211875-0.70580303j -2.4664221 +0.02518135j]
Gradient descent step 122, energy: (-0.11179604336277107-9.252326764532599e-18j)
Params 2: [ 1.39874658+1.24614003j  0.35255417-1.60491574j -3.29529022-0.0523222j
 -3.45055095+0.55547164j  0.33151301-0.96340369j -1.07

Params 2: [ 1.42094866+1.24614003j  0.33584621-1.60491574j -3.37709664-0.0523222j
 -3.48641604+0.55547164j  0.15579658-0.96340369j -1.1099703 -0.18032147j
  0.57540984-1.19050362j  2.93405924+0.21987182j  0.04789212-0.21291913j
  2.05366016-1.41149914j  0.17662624-0.48298102j -1.08821936+1.20176208j
 -2.40045303-0.70580303j -2.48726757+0.02518135j]
Gradient descent step 141, energy: (-0.10771991582371017+7.622930201209518e-18j)
Params 2: [ 1.42212844+1.24614003j  0.33500251-1.60491574j -3.38129458-0.0523222j
 -3.48824467+0.55547164j  0.14683915-0.96340369j -1.11171582-0.18032147j
  0.57699039-1.19050362j  2.93573643+0.21987182j  0.05286502-0.21291913j
  2.0550822 -1.41149914j  0.17534287-0.48298102j -1.08905163+1.20176208j
 -2.40693806-0.70580303j -2.48825117+0.02518135j]
Gradient descent step 142, energy: (-0.10754281026597862+7.5614685925009e-18j)
Params 2: [ 1.42339735+1.24614003j  0.33411011-1.60491574j -3.38554492-0.0523222j
 -3.49008666+0.55547164j  0.1374608 -0.96340369j -1.1135